This notebook trains a logistic regression model, finds the optimal value of C, and reports F1 and log loss scores.

In [1]:
import numpy as np
import pandas as pd
import zipfile
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

In [2]:
# Unzip data files into the "csv" subdirectory 
# (unless you have already done this since running the Data Set Up notebook)

# **IMPORTANT**  This will overwrite existing files in the "csv" folder in your local repo
# with the most recent data files from the data.zip file

# Unzip 80% training data
unzip_training_data = zipfile.ZipFile("data_subset.zip", "r")
unzip_training_data.extractall()
unzip_training_data.close()

# Unzip development and training data
unzip_test_data = zipfile.ZipFile("testing.zip", "r")
unzip_test_data.extractall()
unzip_test_data.close()

# Unzip full set of training data for creating predictions to submit to Kaggle
unzip_all_data = zipfile.ZipFile("data.zip", "r")
unzip_all_data.extractall()
unzip_all_data.close()

In [3]:
# Load these csv files into numpy arrays for testing on development data
train_data = np.loadtxt('csv/train_data.csv', delimiter=",")
train_labels = np.loadtxt('csv/train_labels.csv', dtype=str, delimiter=",")
dev_data = np.loadtxt('csv/dev_data.csv', delimiter=",")
dev_labels = np.loadtxt('csv/dev_labels.csv', dtype=str, delimiter=",")

In [4]:
# Load these csv files into numpy arrays for creating predictions to submit to Kaggle
train_data_all = np.loadtxt('csv/train_data_all.csv', delimiter=",")
train_labels_all = np.loadtxt('csv/train_labels_all.csv', dtype=str, delimiter=",")
test_data_all = np.loadtxt('csv/test_data_all.csv', delimiter=",")

In [5]:
# print shapes to compare before and after csv conversion
print("train_data shape is", train_data.shape)
print("train_labels shape is", train_labels.shape)
print("dev_data shape is", dev_data.shape)
print("dev_labels shape is", dev_labels.shape)

('train_data shape is', (702439L, 58L))
('train_labels shape is', (702439L,))
('dev_data shape is', (175610L, 58L))
('dev_labels shape is', (175610L,))


In [6]:
print("train_data_all shape is", train_data_all.shape)
print("train_labels_all shape is", train_labels_all.shape)
print("test_data_all shape is", test_data_all.shape)

('train_data_all shape is', (878049L, 58L))
('train_labels_all shape is', (878049L,))
('test_data_all shape is', (884262L, 58L))


In [7]:
# Set up functions for training logistic regression model and finding optimal value of C

def TrainLR(data, labels, test_data, C_value=1.0):
    """This function takes in training data and labels, testing data,
    and can accept different values of C (the learning rate).
    It trains a logistic regression model and returns the model and predicted probabilities.
    """
    LR = LogisticRegression(C=C_value, n_jobs = -1)
    LR.fit(data, labels)
    pp = LR.predict_proba(test_data)
    return LR, pp

def find_C(data, labels, dev_data, dev_labels, C_values):
    """Find optimal value of C in a logistic regression model.  
    
    Note that this cannot be used on test data from Kaggle 
    because we do not have labels for that data.  This function is intended to only be used
    in the development stage with the development data.
    """
    for C in C_values:      
        LR, pp = TrainLR(data, labels, dev_data, C, n_jobs = -1)
        predictions = LR.predict(dev_data)
        f1 = metrics.f1_score(dev_labels, predictions, average = "weighted")
        logloss = metrics.log_loss(dev_labels, pp)
        
        # Print F1 score and log loss for each value of k
        print("For C =", C, "the F1 score is", round(f1, 6), "and the Log Loss score is", round(logloss, 6))
    print("\n")

In [8]:
# IF there are additional changes to make to the data for this model
# that would be easier to do in pandas, uncomment and run this code. 
# This model works the same whether the data is in numpy or pandas, so presumably so do other models

#train_data = pd.DataFrame(train_data)
#train_labels = pd.DataFrame(train_labels)
#dev_data = pd.DataFrame(dev_data)
#dev_labels = pd.DataFrame(dev_labels)
#train_data_all = pd.DataFrame(train_data_all)
#train_labels_all = pd.DataFrame(train_labels_all)
#test_data_all = pd.DataFrame(test_data_all)

print(np.mean(test_data_all))

-1.73167566453e-11


In [ ]:
# Find the optimal value of C using the 80% training data and the development data
C_values = [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0, 100.00, 1000.0]
find_C(train_data, train_labels, dev_data, dev_labels, C_values)

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


For C = 0.0001 the F1 score is 0.148899 and the Log Loss score is 3.011221
For C = 0.001 the F1 score is 0.152004 and the Log Loss score is 2.632073
For C = 0.01 the F1 score is 0.152955 and the Log Loss score is 2.544749
For C = 0.1 the F1 score is 0.153044 and the Log Loss score is 2.536512


In [14]:
# Train model with a single value of C with 80% training data and development data
C_value = 2.0
LR, pp = TrainLR(train_data, train_labels, dev_data, C_value)
logloss = metrics.log_loss(dev_labels, pp)
print(logloss)

/Users/laura/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


2.54267758117


In [9]:
# Before submitting to Kaggle, run the model on the full set of training data and test data
# using the optimal value for the model
C_value = 2.0
LR, pp = TrainLR(train_data_all, train_labels_all, test_data_all, C_value)

In [10]:
# Set up predictions for submission to Kaggle
headers = ["ARSON","ASSAULT","BAD CHECKS","BRIBERY","BURGLARY","DISORDERLY CONDUCT","DRIVING UNDER THE INFLUENCE",
           "DRUG/NARCOTIC","DRUNKENNESS","EMBEZZLEMENT","EXTORTION","FAMILY OFFENSES","FORGERY/COUNTERFEITING",
           "FRAUD","GAMBLING","KIDNAPPING","LARCENY/THEFT","LIQUOR LAWS","LOITERING","MISSING PERSON","NON-CRIMINAL",
           "OTHER OFFENSES","PORNOGRAPHY/OBSCENE MAT","PROSTITUTION","RECOVERED VEHICLE","ROBBERY","RUNAWAY",
           "SECONDARY CODES","SEX OFFENSES FORCIBLE","SEX OFFENSES NON FORCIBLE","STOLEN PROPERTY","SUICIDE",
           "SUSPICIOUS OCC","TREA","TRESPASS","VANDALISM","VEHICLE THEFT","WARRANTS","WEAPON LAWS"]
data = pd.DataFrame(data=pp, 
                    index=[x for x in range(len(test_data_all))], 
                    columns=headers)
data.columns.name ="Id"
print(data.shape)
print(data)

(884262, 39)
Id         ARSON   ASSAULT  BAD CHECKS   BRIBERY  BURGLARY  \
0       0.011413  0.132513    0.000007  0.001573  0.033707   
1       0.010538  0.126437    0.000006  0.001534  0.024041   
2       0.003258  0.087976    0.000011  0.000386  0.054249   
3       0.005019  0.116112    0.000005  0.001531  0.020660   
4       0.005019  0.116112    0.000005  0.001531  0.020660   
5       0.004678  0.102300    0.000005  0.000805  0.014835   
6       0.005589  0.120153    0.000010  0.001376  0.056190   
7       0.005033  0.113021    0.000009  0.001291  0.041263   
8       0.002683  0.102762    0.000007  0.001217  0.024580   
9       0.001612  0.057956    0.000006  0.000204  0.011200   
10      0.005135  0.112459    0.000011  0.001215  0.058702   
11      0.002489  0.098141    0.000006  0.001206  0.016913   
12      0.002489  0.098141    0.000006  0.001206  0.016913   
13      0.003073  0.090870    0.000016  0.000321  0.051168   
14      0.001703  0.059428    0.000008  0.000194  0.01893

Create zipped csv file for Kaggle
#### Update the filename first in all lines of the following code
Add something unique after our names to avoid overwriting other submission files

In [11]:
data.to_csv('Williams_Gascoigne_Vignola_Regression4.csv', index_label = "Id")

In [12]:
zip_probs = zipfile.ZipFile("Williams_Gascoigne_Vignola_Regression4.zip", "w")
zip_probs.write("Williams_Gascoigne_Vignola_Regression4.csv", compress_type=zipfile.ZIP_DEFLATED)
zip_probs.close()

### Results from previous datasets and/or model parameters

**First Submission**   
Results on development data from dataset as of Saturday 11/18, with weather added, latitude outliers removed, binarized and normalized features:

For C = 0.0001 the F1 score is 0.147075 and the Log Loss score is 3.014795  
For C = 0.001 the F1 score is 0.150284 and the Log Loss score is 2.638404  
For C = 0.01 the F1 score is 0.151366 and the Log Loss score is 2.551881  
For C = 0.1 the F1 score is 0.151589 and the Log Loss score is 2.543797  
For C = 0.5 the F1 score is 0.151615 and the Log Loss score is 2.543427  
For C = 1.0 the F1 score is 0.151579 and the Log Loss score is 2.543396  
**For C = 2.0 the F1 score is 0.151605 and the Log Loss score is 2.543383**  
For C = 10.0 the F1 score is 0.151657 and the Log Loss score is 2.543385  
For C = 100.0 the F1 score is 0.151619 and the Log Loss score is 2.543447  
For C = 1000.0 the F1 score is 0.151616 and the Log Loss score is 2.543544  

Predictions on test data from training on full data set are in zip file ending with Regression1

Kaggle score from that zip file that we thought should have correlated with the above scores on dev data was 18.20988 (!?)  


**Second Submission**  
We changed our workflow along the way, so our first step was to re-run this notebook to confirm that we are unzipping and using the latest version of the data, in particular the full set of training and test data. 

Log loss on dev data after this step, with C=2.0 is 2.54338  
Predictions from this step are in zip file ending with Regression2  
Kaggle score is the same: 18.20989   
So the problem ended up being our test data was not normalized.


**Third Submission**  
On Cyprian's suggestion, we added the multi_class = 'multinomial' argument to the logistic regression model, because we don't have a binary output variable.  

Log loss on dev data after this step, with C=2.0 is 2.54267  
Predictions from this step are in zip file ending with Regression3  
Kaggle score on this set of predictions is:  33.37633   
This is even worse! Removed the multi_class argument from the model code.

**Fourth Submission**
Fixed a critical bug in submissions 2 and 3 that basically made them nonsense. Our test data was not normalized and now it is.
